In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
!pip install --upgrade nbformat
!pip install --upgrade nbconvert
print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.5.4
    Uninstalling nbconvert-6.5.4:
      Successfully uninstalled nbconvert-6.5.4
2.18.0-dev20240817


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-08-18 00:50:20--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2024-08-18 00:50:20 (23.3 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-08-18 00:50:20--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
df_train = pd.read_csv(train_file_path, sep = '\t', header = None, names = ['Class', 'Message'])
df_test = pd.read_csv(test_file_path, sep = '\t', header = None, names = ['Class', 'Message'])

In [4]:
print(df_train.shape)
print(df_test.shape)

(4179, 2)
(1392, 2)


In [5]:
df_train.head(10)

,Class,Message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
5,ham,in xam hall boy asked girl tell me the startin...
6,ham,genius what's up. how your brother. pls send h...
7,ham,they finally came to fix the ceiling.
8,spam,urgent! call 09066350750 from your landline. y...
9,ham,now that you have started dont stop. just pray...


In [6]:
df_test.head(10)

,Class,Message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
5,ham,your opinion about me? 1. over 2. jada 3. kusr...
6,ham,yesterday its with me only . now am going home.
7,ham,yes. it's all innocent fun. o:-)
8,ham,a boy was late 2 home. his father: power of fr...
9,ham,is ur changes 2 da report big? cos i've alread...


In [7]:
print(df_train.isna().sum())
print(df_test.isna().sum())

Class      0
Message    0
dtype: int64
Class      0
Message    0
dtype: int64


In [8]:
print(df_train.dtypes)
print(df_test.dtypes)

Class      object
Message    object
dtype: object
Class      object
Message    object
dtype: object


In [9]:
# Create dummy variables

df_train = pd.get_dummies(df_train, columns = ['Class'], drop_first = True)
df_test = pd.get_dummies(df_test, columns = ['Class'], drop_first = True)
print(df_train.head())
print(df_test.head())

                                             Message  Class_spam
0  ahhhh...just woken up!had a bad dream about u ...       False
1                           you can never do nothing       False
2  now u sound like manky scouse boy steve,like! ...       False
3  mum say we wan to go then go... then she can s...       False
4  never y lei... i v lazy... got wat? dat day ü ...       False
                                             Message  Class_spam
0  i am in hospital da. . i will return home in e...       False
1         not much, just some textin'. how bout you?       False
2  i probably won't eat at all today. i think i'm...       False
3  don‘t give a flying monkeys wot they think and...       False
4                                who are you seeing?       False


In [10]:
df_train['Class_spam'] = df_train['Class_spam'].astype(int)
df_test['Class_spam'] = df_test['Class_spam'].astype(int)
print(df_train.head())
print(df_test.head())

                                             Message  Class_spam
0  ahhhh...just woken up!had a bad dream about u ...           0
1                           you can never do nothing           0
2  now u sound like manky scouse boy steve,like! ...           0
3  mum say we wan to go then go... then she can s...           0
4  never y lei... i v lazy... got wat? dat day ü ...           0
                                             Message  Class_spam
0  i am in hospital da. . i will return home in e...           0
1         not much, just some textin'. how bout you?           0
2  i probably won't eat at all today. i think i'm...           0
3  don‘t give a flying monkeys wot they think and...           0
4                                who are you seeing?           0


In [11]:
# Data processing

def text_processing(df):

  import string
  import re

  # Eliminate punctuation signs and number

  punctuation = string.punctuation
  spaces = ' ' * len(punctuation)
  df['Message'] = df['Message'].apply(lambda x: x.translate(str.maketrans(punctuation, spaces)))
  df['Message'] = df['Message'].apply(lambda x: re.sub(r'\d+', '', x))

  # Tokenize the messages

  df['Message'] = df['Message'].astype(str)
  df['Message'] = df['Message'].apply(lambda x: x.split())

  return df

In [12]:
df_train = text_processing(df_train)
df_test = text_processing(df_test)

In [13]:
df_train.head()

,Message,Class_spam
0,"[ahhhh, just, woken, up, had, a, bad, dream, a...",0
1,"[you, can, never, do, nothing]",0
2,"[now, u, sound, like, manky, scouse, boy, stev...",0
3,"[mum, say, we, wan, to, go, then, go, then, sh...",0
4,"[never, y, lei, i, v, lazy, got, wat, dat, day...",0


In [14]:
df_test.head()

,Message,Class_spam
0,"[i, am, in, hospital, da, i, will, return, hom...",0
1,"[not, much, just, some, textin, how, bout, you]",0
2,"[i, probably, won, t, eat, at, all, today, i, ...",0
3,"[don‘t, give, a, flying, monkeys, wot, they, t...",0
4,"[who, are, you, seeing]",0


In [15]:
# Create the vocabulary

vocabulary = [word for message in df_train['Message'] for word in message]
vocabulary = list(set(vocabulary))
total_words = len(vocabulary) + 1
print(total_words)

6782


In [16]:
# Getting the max length of a message

max_length = max(len(message) for message in df_train['Message'])
print(max_length)

190


In [17]:
# Create an encoder dictionary

encoder_dict = {}

for word in vocabulary:
    if word not in encoder_dict:
        encoder_dict[word] = len(encoder_dict) + 1

print(encoder_dict)

{'secs': 1, 'showers': 2, 'catching': 3, 'yam': 4, 'looking': 5, 'largest': 6, 'drunkard': 7, 'hm': 8, 'nvm': 9, 'argh': 10, 'arng': 11, 'ask': 12, 'wearing': 13, 'gone': 14, 'cn': 15, 'lady': 16, 'famamus': 17, 'wrong': 18, 'worth': 19, 'usually': 20, 'fish': 21, 'that\x92s': 22, 'callcostppmmobilesvary': 23, 'webadres': 24, 'joker': 25, 'flaked': 26, 'canlove': 27, 'simpsons': 28, 'yck': 29, 'gei': 30, 'profit': 31, 'like': 32, 'bbdeluxe': 33, 'jap': 34, 'gary': 35, 'excused': 36, 'picked': 37, 'never': 38, 'comwinppmxagesubscription': 39, 'devouring': 40, 'tahan': 41, 'vibrant': 42, 'under': 43, 'reminder': 44, 'eek': 45, 'reveal': 46, 'sweets': 47, 'alaikkum': 48, 'y': 49, 'superb': 50, 'western': 51, 'sending': 52, 'outs': 53, 'whore': 54, 'flag': 55, 'tsandcs': 56, 'lshb': 57, 'phrase': 58, 'mag': 59, 'cruel': 60, 'lol': 61, 'forwarded': 62, 'varunnathu': 63, 'nothing': 64, 'exciting': 65, 'gimme': 66, 'settle': 67, 'somebody': 68, 'male': 69, 'hiya': 70, 'monos': 71, 'respectful

In [18]:
def encode(df, encoder):
  df['Message'] = df['Message'].apply(lambda x: [encoder.get(word, -1) for word in x])
  return df

In [19]:
encode(df_train, encoder_dict)
encode(df_test, encoder_dict)
print(df_train.head())
print(df_test.head())

                                             Message  Class_spam
0  [4239, 5260, 3759, 3007, 2085, 5015, 6708, 433...           0
1                         [2427, 4770, 38, 1145, 64]           0
2  [1097, 5409, 1610, 32, 1882, 5747, 4790, 4888,...           0
3  [730, 4565, 5135, 1198, 5255, 4455, 1907, 4455...           0
4  [38, 49, 5694, 4243, 2090, 3770, 3898, 6730, 2...           0
                                             Message  Class_spam
0  [4243, 2879, 1940, 3267, 5285, 4243, 3185, 615...           0
1       [348, 4616, 5260, 5812, -1, 890, 4490, 2427]           0
2  [4243, 2276, 2615, 406, 1721, 3020, 6651, 4999...           0
3  [5570, 4091, 5015, -1, -1, 5832, 4057, 2944, 1...           0
4                           [5493, 3147, 2427, 5261]           0


In [20]:
# Create train and test labels and convert them to numpy arrays

train_labels = df_train.pop('Class_spam').to_numpy()
test_labels = df_test.pop('Class_spam').to_numpy()


df_train = np.squeeze(df_train.to_numpy())
df_test = np.squeeze(df_test.to_numpy())

print(df_train)
print(df_test)
print(train_labels)
print(test_labels)

[list([4239, 5260, 3759, 3007, 2085, 5015, 6708, 4334, 1428, 5409, 5326, 806, 4243, 2250, 32, 5409, 5689, 1097, 4243, 939, 2159, 5843, 1428, 5975, 2909, 1088, 4243, 4998, 2350, 3007, 4626, 2322])
 list([2427, 4770, 38, 1145, 64])
 list([1097, 5409, 1610, 32, 1882, 5747, 4790, 4888, 32, 4243, 1690, 2915, 1858, 5285, 3837, 1424, 5832, 5128, 5409, 3134, 1962, 2338, 817])
 ...
 list([448, 3081, 520, 3066, 4520, 1957, 6118, 5260, 6755, 300, 5767, 2823, 5255, 1097, 406, 2824, 1672, 4246, 3274, 6189])
 list([4892, 5014, 3472, 5010, 1690, 5265, 2427, 5255, 5716, 369, 2961, 3909, 2288, 883, 1585, 6224, 369, 1672, 3664, 5062, 5409, 3472, 5014, 3169, 5014, 2010, 5255])
 list([4391, 5144, 6708, 4834, 3887, 5409, 1040, 990, 1940, 4676, 4243, 2070, 2743, 5255, 6150, 1940, 6672, 2408, 4243, 2070, 3117, 6245, 5144, 5394, 6006, 1734, 3714, 4626, 5137, 3909, 5716, 6549, 4100, 935])]
[list([4243, 2879, 1940, 3267, 5285, 4243, 3185, 6158, 1424, 1940, 2071])
 list([348, 4616, 5260, 5812, -1, 890, 4490, 242

In [21]:
print(df_train[0])

[4239, 5260, 3759, 3007, 2085, 5015, 6708, 4334, 1428, 5409, 5326, 806, 4243, 2250, 32, 5409, 5689, 1097, 4243, 939, 2159, 5843, 1428, 5975, 2909, 1088, 4243, 4998, 2350, 3007, 4626, 2322]


In [22]:
# Data homogeneity

data_length = max_length // 2
from keras.preprocessing import sequence

df_train = sequence.pad_sequences(df_train, data_length)
df_test = sequence.pad_sequences(df_test, data_length)

In [23]:
print(df_train[0])
print(df_test[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 4239 5260 3759 3007 2085 5015 6708
 4334 1428 5409 5326  806 4243 2250   32 5409 5689 1097 4243  939 2159
 5843 1428 5975 2909 1088 4243 4998 2350 3007 4626 2322]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
 4243 2879 1940 3267 5285 4243 3185 6158 1424 1940 2071]


In [24]:
# Building the model

model = keras.Sequential([
    keras.layers.Embedding(total_words, 62),
    keras.layers.LSTM(62),
    keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Training the model

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
history = model.fit(df_train, train_labels, epochs = 10)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - acc: 0.8774 - loss: 0.3405
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - acc: 0.9895 - loss: 0.0404
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - acc: 0.9934 - loss: 0.0203
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - acc: 0.9988 - loss: 0.0084
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - acc: 0.9994 - loss: 0.0043
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - acc: 0.9991 - loss: 0.0038
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - acc: 0.9997 - loss: 0.0016
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - acc: 0.9991 - loss: 0.0027
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - acc: 0.9999 - loss: 8.9526e-04
Epoch 10/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step - acc: 0.9994 - loss: 0.0042


In [26]:
results = model.evaluate(df_test, test_labels)
print(results)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9862 - loss: 0.0599
[0.05059829354286194, 0.9892241358757019]


In [27]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):

  if isinstance(pred_text, str):
    pred_text = [pred_text]

  predictions_list = []
  for text in pred_text:
    prediction = []

    # Data preprocessing

    data_dict = {'Message': [text]}
    data = pd.DataFrame(data_dict)

    data = text_processing(data)
    data = encode(data, encoder_dict)
    data = np.squeeze(data.to_numpy())
    data = np.expand_dims(data, axis = 0)
    data = sequence.pad_sequences(data, data_length)

    pred = model.predict(data)
    prediction.append(pred[0][0])

    if pred[0][0] <= 0.5:
      label = "ham"
      prediction.append(label)
    else:
      label = "spam"
      prediction.append(label)

    predictions_list.append(prediction)

  predictions = []

  for list_value in predictions_list:
    for element in list_value:
      predictions.append(element)

  return predictions

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
[8.384911e-05, 'ham']


In [28]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
You passed the challenge. Great job!
